In [2]:
%load_ext autoreload
%autoreload 2

In [22]:
import pickle
import pandas as pd 
import numpy as np
import torch
import os
from math import ceil
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
from rdkit.Geometry import Point3D
from openbabel import pybel
import re

In [3]:
from utils_eval import build_pdb_dict, center2zero, pp_match
from pp_matching_specific_site import compute_center

In [4]:
import py3Dmol
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG, Image
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.ipython_useSVG = True
IPythonConsole.molSize = 600, 600

In [5]:
ligand_name = '1oty_A_rec_3occ_dih_lig_tt_docked_18'  # No 3, 17 are alright

In [6]:
bridge_type = 'vp'
root_path = os.path.join('structure_based', bridge_type, ligand_name)
# root_path = os.path.join('ligand_based', ligand_name)
aromatic = True
gen_path = os.path.join(root_path, 'aromatic' if aromatic else 'basic')

optimization = False
gen_path += '_optimized' if optimization else ''

In [7]:
score_file = os.path.join(root_path, 'aromatic_matches.pkl' if aromatic else 'basic_matches.pkl')
with open(score_file, 'rb') as f:
    matches = pickle.load(f)
matches

{'3occ_89': array([1, 1, 1, 0]),
 '3occ_27': array([1, 1, 1, 0]),
 '3occ_42': array([1, 1, 1, 0]),
 '3occ_53': array([1, 1, 1, 0]),
 '3occ_44': array([1, 1, 1, 0]),
 '3occ_81': array([1, 1, 1, 0]),
 '3occ_18': array([1, 0, 1, 0]),
 '3occ_91': array([1, 1, 1, 0]),
 '3occ_56': array([1, 1, 1, 0]),
 '3occ_69': array([1, 1, 1, 0]),
 '3occ_52': array([1, 1, 1, 0]),
 '3occ_61': array([1, 0, 1, 1]),
 '3occ_78': array([1, 1, 1, 0]),
 '3occ_67': array([1, 1, 1, 0]),
 '3occ_79': array([1, 1, 1, 0]),
 '3occ_31': array([1, 1, 1, 0]),
 '3occ_97': array([1, 1, 1, 0]),
 '3occ_30': array([1, 1, 1, 0]),
 '3occ_51': array([1, 1, 1, 0]),
 '3occ_38': array([1, 1, 1, 0]),
 '3occ_25': array([1, 1, 1, 0]),
 '3occ_37': array([1, 1, 1, 0]),
 '3occ_29': array([1, 1, 1, 0]),
 '3occ_16': array([1, 1, 1, 0]),
 '3occ_26': array([0, 1, 1, 0]),
 '3occ_41': array([1, 1, 1, 0]),
 '3occ_71': array([1, 1, 1, 0]),
 '3occ_48': array([1, 1, 1, 0]),
 '3occ_34': array([1, 1, 1, 0]),
 '3occ_20': array([1, 1, 1, 0]),
 '3occ_23'

In [16]:
test_data = torch.load(os.path.join('structure_based', ligand_name, ligand_name + '_aromatic' + '.pt' if aromatic else ligand_name + '.pt'))
center = compute_center(test_data['target_pos'], test_data['Gt_mask']).numpy()

# with open(os.path.join('structure_based', ligand_name, 'pp_info.pkl'), 'rb') as f:
#     pp_info = pickle.load(f)

# pp_info['pp_positions'] = pp_info['pp_positions'] - center
# pp_info['pp_positions']

In [17]:
raw_data_path = '../../data/cleaned_crossdocked_data/raw'
pdb_dict, pdb_rev_dict = build_pdb_dict(raw_data_path)

In [18]:
lig_id = ligand_name[ligand_name.rfind('rec')+4:ligand_name.rfind('rec')+8]

In [25]:
idx = '17'
lig_file = os.path.join(gen_path, lig_id + '_' + idx + '.sdf')
lig_file

'structure_based/vp/1oty_A_rec_3occ_dih_lig_tt_docked_18/aromatic/3occ_17.sdf'

In [26]:
rdmol = Chem.MolFromMolFile(lig_file, sanitize=True)
conf = rdmol.GetConformer()
atom_positions = conf.GetPositions()

atom_positions

array([[ 0.4156,  0.1187,  0.1715],
       [ 1.8797, -0.1086,  1.9821],
       [ 2.6091, -0.0277,  2.5366],
       [ 2.5977,  1.5309,  0.3647],
       [ 0.2768, -0.213 ,  0.773 ],
       [ 2.4895, -0.804 ,  3.4779],
       [ 0.9812, -1.3404,  3.036 ],
       [ 0.9693, -0.8582,  2.2197],
       [ 1.496 ,  0.592 ,  0.7376],
       [-0.1006, -0.9306,  1.4903],
       [ 1.739 , -1.376 ,  3.6523],
       [-2.085 ,  0.839 , -2.8666],
       [-1.533 ,  0.5665, -1.8536],
       [-2.9137,  0.7487, -3.6254],
       [-2.57  ,  0.655 , -4.9382],
       [-1.0113,  0.2729, -3.0465],
       [-0.0424,  0.0074, -0.9688],
       [-1.716 ,  0.842 , -0.571 ],
       [-3.1951,  0.8439, -2.5512]])

In [27]:
for i in range(rdmol.GetNumAtoms()):
    x,y,z = atom_positions[i] + center
    print(x, y, z)
    conf.SetAtomPosition(i,Point3D(x,y,z))

137.2726556640625 101.7095432006836 39.33543280029297
138.7367556640625 101.4822432006836 41.14603280029297
139.4661556640625 101.5631432006836 41.70053280029297
139.4547556640625 103.1217432006836 39.52863280029297
137.1338556640625 101.3778432006836 39.93693280029297
139.3465556640625 100.78684320068359 42.64183280029297
137.8382556640625 100.25044320068359 42.19993280029297
137.8263556640625 100.7326432006836 41.38363280029297
138.3530556640625 102.18284320068359 39.90153280029297
136.7564556640625 100.6602432006836 40.65423280029297
138.5960556640625 100.21484320068359 42.816232800292966
134.7720556640625 102.42984320068359 36.29733280029297
135.3240556640625 102.1573432006836 37.31033280029297
133.9433556640625 102.3395432006836 35.53853280029297
134.2870556640625 102.2458432006836 34.22573280029297
135.8457556640625 101.8637432006836 36.11743280029297
136.8146556640625 101.5982432006836 38.19513280029297
135.1410556640625 102.43284320068359 38.59293280029297
133.6619556640625 102

In [28]:
with Chem.SDWriter(os.path.join('structure_based', ligand_name, lig_id + '_' + idx + '_2pocket.sdf')) as w:
    w.write(rdmol)